In [1]:
import torch

# Tensor

In [4]:
tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])
tensor3d = torch.tensor([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])

In [5]:
tensor0d, tensor1d, tensor2d, tensor3d

(tensor(1),
 tensor([1, 2, 3]),
 tensor([[1, 2, 3],
         [4, 5, 6]]),
 tensor([[[ 1,  2,  3],
          [ 4,  5,  6]],
 
         [[ 7,  8,  9],
          [10, 11, 12]]]))

In [6]:
tensor1d.dtype

torch.int64

In [7]:
tensor2d.shape

torch.Size([2, 3])

In [8]:
tensor3d.shape

torch.Size([2, 2, 3])

In [9]:
tensor2d.reshape(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [ ]:
# more commonly used than reshape
tensor2d.view(3, 2)

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [12]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [13]:
tensor2d.matmul(tensor2d.T)

tensor([[14, 32],
        [32, 77]])

In [14]:
# alternatively, you can use the @ operator
tensor2d @ tensor2d.T

tensor([[14, 32],
        [32, 77]])

# Models as computation graphs

In [15]:
import torch.nn.functional as F

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2])
b = torch.tensor([0.0])

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)
print(loss)

tensor(0.0852)


# Automatic differentiation made easy

In [ ]:
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

# retain_graph=True allows us to compute multiple gradients without recomputing the forward pass
grad_L_w1 = grad(loss, w1, retain_graph=True) 
grad_L_b = grad(loss, b, retain_graph=True) 

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [22]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


# Implementing multilayer neural networks